## import

In [1]:
%load_ext autoreload
%autoreload 2

In [8]:
import torch
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
from models.utils import *

In [11]:
tde = TD_Encoder(16)
td = 1.0* torch.ones(size=(1024, 1))
out = tde(td)
out.shape

torch.Size([1024, 16])

In [12]:
out[0], out[1]

(tensor([-0.2439,  0.1176, -0.0426,  0.0015, -0.1101,  0.1576, -0.0535, -0.0309,
          0.0995,  0.0938,  0.0742,  0.0276, -0.0276, -0.0258, -0.1131, -0.0764],
        grad_fn=<SelectBackward0>),
 tensor([-0.2439,  0.1176, -0.0426,  0.0015, -0.1101,  0.1576, -0.0535, -0.0309,
          0.0995,  0.0938,  0.0742,  0.0276, -0.0276, -0.0258, -0.1131, -0.0764],
        grad_fn=<SelectBackward0>))

In [15]:
r = out[:,-1]
r.shape

torch.Size([1024])

In [17]:
ro = torch.cat((out, r.unsqueeze(-1)), dim=-1)
ro.shape

torch.Size([1024, 17])

In [2]:
import torch
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.data import Data
from torch_cluster import knn_graph, knn
from torch import Tensor
from typing import Optional, Tuple
import torch_geometric.transforms as T
from torch_geometric.nn import MLP, PointNetConv, fps, global_max_pool, radius, global_mean_pool
import os
from torch.utils.data import Dataset
import glob

from torch_geometric.data import Data
import torch_geometric.transforms as T
# from torch_geometric.loader import DataLoader
from torch_geometric.nn import GATv2Conv


## oth test

## mp 加法

In [12]:
#demo data
bpos_1 = torch.randn(2*1024,3)
bpos_5 =torch.randn(2*1024,3)
bfea_1 = torch.randn(2*1024,24)
bfea_5 =torch.randn(2*1024,24)
t = torch.LongTensor([0,1])
dbatch = torch.repeat_interleave(t, 1024)


In [13]:
# Finds for each element in :obj:`y` the :obj:`k` nearest points in :obj:`x`.
index_ex = knn(x=bpos_1,y=bpos_5, k=10, batch_x=dbatch, batch_y=dbatch)
index_ex


tensor([[   0,    0,    0,  ..., 2047, 2047, 2047],
        [ 674,  691,  514,  ..., 1390, 1777, 1802]])

In [14]:
index_in = knn_graph(bpos_5, k=10, batch=dbatch, loop=False)
index_in

tensor([[ 417,  438,  384,  ..., 1115, 1155, 1887],
        [   0,    0,    0,  ..., 2047, 2047, 2047]])

In [15]:
in_id_source, in_id_target =  index_in[0], index_in[1]

ex_id_source, ex_id_target =  index_ex[1], index_ex[0]

In [34]:
class Local_Point_Trans(nn.Module):
    def __init__(self) -> None:
        super().__init__()
        self.lin_p = nn.Linear(4, 256)
        self.lin_q = nn.Linear(256, 256)
        self.lin_k = nn.Linear(256, 256)
        self.lin_v = nn.Linear(256, 256)
        self.lin_w = nn.Linear(256, 256)
        self.att = nn.Softmax(dim=1)


    def forward(self, fea_j, fea_i, xyzt_j, xyzt_i):
        """
        fea_j: [N, k, 256]
        fea_i: [N, k, 256]
        xyzt_j: [N, k, 4]
        xyzt_i: [N, k, 4]
        """ 

        pe = self.lin_p(xyzt_j - xyzt_i)  # [N, k, 256]

        w = self.lin_w(self.lin_q(fea_i) - self.lin_k(fea_j) + pe) # [N, k, 256]

        w = self.att(w)

        print(w.sum(dim=1)[0]) #[1,1,...,1] 256dim

        v = self.lin_v(fea_j) + pe

        print(v.shape)
        print(w.shape)

        return w, v





In [35]:
fea_i = torch.randn(100,20,256)
xyzt_i = torch.randn(100,20, 4)
fea_j = torch.randn(100,20, 256)
xyzt_j = torch.randn(100,20, 4)

lpt = Local_Point_Trans()
w, v = lpt(fea_j, fea_i, xyzt_j, xyzt_i)


tensor([1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000,
        1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000,
        1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000,
        1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000,
        1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000,
        1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000,
        1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000,
        1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000,
        1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000,
        1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000,
        1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000,
        1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000,
        1.0000, 1.0000, 1.0000, 1.0000, 

In [33]:
bpos_1 = torch.randn(2*1024,3)
bpos_5 =torch.randn(2*1024,3)
bfea_1 = torch.randn(2*1024,24)
bfea_5 =torch.randn(2*1024,24)
t = torch.LongTensor([0,1])
dbatch = torch.repeat_interleave(t, 1024)

## fps test

In [ ]:
seed = 42
torch.manual_seed(seed)
def original_fps(xyz, npoint):
    device = xyz.device
    B, N, C = xyz.shape
    centroids = torch.zeros(B, npoint, dtype=torch.long).to(device)
    distance = torch.ones(B, N).to(device) * 1e10
    farthest = torch.randint(0, N, (B,), dtype=torch.long).to(device)
    batch_indices = torch.arange(B, dtype=torch.long).to(device)
    for i in range(npoint):
        centroids[:, i] = farthest
        centroid = xyz[batch_indices, farthest, :].view(B, 1, 3)
        dist = torch.sum((xyz - centroid) ** 2, -1)
        mask = dist < distance
        distance[mask] = dist[mask]
        farthest = torch.max(distance, -1)[1]
    return centroids


def new_fps(xyz, npoint):
    device = xyz.device
    B, N, C = xyz.shape

    ptr = torch.repeat_interleave(
        torch.tensor([N*i for i in range(B)],requires_grad=False, dtype=torch.long), 
        npoint
    ).to(device)
    batch_x = torch.repeat_interleave(torch.tensor(range(B),requires_grad=False, dtype=torch.long), 
                                      N).to(device)
    
    ratio_x = float(npoint/N)

    x = xyz.reshape(-1,C)

    x_idx = fps(x, batch_x, ratio_x)
    x_idx = x_idx - ptr

    centroids = x_idx.reshape(B, npoint).long().to(device)
    return centroids



In [ ]:
seed = 42
torch.manual_seed(seed)

bpos_1 = torch.randn(2,40,3)
n_p = 10

ce_o = original_fps(bpos_1, n_p)
ce_n = new_fps(bpos_1, n_p)
ce_o, ce_n

(tensor([[ 1, 15, 11,  5, 27,  7, 32, 22, 10, 31],
         [ 6, 24, 31, 34, 20,  8, 37, 14,  2, 12]]),
 tensor([[ 9, 32, 30, 13,  0, 38,  2, 20, 26,  3],
         [ 2, 12,  3, 32, 16, 18, 30, 24, 33,  6]]))

## pointnet++

In [9]:
from dataset.kitti_dataset_v2 import KittiDataset_2
data_config = {'root': "/home/stud/ding/PC_FC/PC_forecasting/kittiraw/dataset/sequences", 'npoints': 4096*4, 'input_num': 5, 'pred_num': 5, 'tr_seqs': ['00']}
demo_dataset = KittiDataset_2(root=data_config['root'], npoints=data_config['npoints'], input_num=data_config['input_num'], pred_num=data_config['pred_num'], seqs=data_config['tr_seqs'])
in_xyz_list, in_fea_list, gt_xyz_list = demo_dataset[0]

In [10]:
in_xyz_list[0].shape, in_fea_list[0].shape,gt_xyz_list[0].shape

(torch.Size([3, 16384]), torch.Size([1, 16384]), torch.Size([3, 16384]))

In [11]:
from torch.utils.data import DataLoader

train_dataloader = DataLoader(demo_dataset, batch_size=2) # collate_fn=custom_collate_fn


In [12]:
bin_xyz, bin_fea, bgt_xyz = next(iter(train_dataloader))
xyz, fea = bin_xyz[0], bin_fea[0]
xyz.shape, fea.shape


(torch.Size([2, 3, 16384]), torch.Size([2, 1, 16384]))

In [13]:
from models.encoder_pn2 import *
pnt_encoder = PNT_2()

In [14]:
en_xyz, en_fea = pnt_encoder(xyz, fea)
en_xyz.shape, en_fea.shape

(torch.Size([2, 3, 1024]), torch.Size([2, 256, 1024]))

In [29]:
aa = en_fea.permute(0,2,1).reshape(-1,256)

aa.reshape(2,-1,256).permute(0,2,1).shape

torch.Size([2, 256, 1024])

In [25]:
bpos_1 = torch.randn(2*1024,3)
bpos_5 =torch.randn(2*1024,3)
bfea_1 = torch.randn(2*1024,256)
bfea_5 =torch.randn(2*1024,256)

In [26]:
t = torch.LongTensor([0,1])
dbatch = torch.repeat_interleave(t, 1024)
by_to_x_index = knn(x=bpos_1,y=bpos_5, k=4, batch_x=dbatch, batch_y=dbatch)
by_to_x_index

tensor([[   0,    0,    0,  ..., 2047, 2047, 2047],
        [ 177,  760,  297,  ..., 1173, 1834, 1262]])

In [27]:
gat_btw_fr = GATv2Conv(in_channels=(256,256),out_channels=256, heads=4, concat=False, negative_slope=0.2, bias=False, add_self_loops=False, flow='target_to_source')
out = gat_btw_fr((bfea_5, bfea_1),by_to_x_index)
out.shape

torch.Size([2048, 256])

## test

## models

In [6]:
class ScaledDotProductAttention(nn.Module):
    """
    Scaled Dot-Product Attention proposed in "Attention Is All You Need"
    Compute the dot products of the query with all keys, divide each by sqrt(dim),
    and apply a softmax function to obtain the weights on the values

    Args: dim, mask
        dim (int): dimention of attention
        mask (torch.Tensor): tensor containing indices to be masked

    Inputs: query, key, value, mask
        - **query** (batch, q_len, d_model): tensor containing projection vector for decoder.
        - **key** (batch, k_len, d_model): tensor containing projection vector for encoder.
        - **value** (batch, v_len, d_model): tensor containing features of the encoded input sequence.
        - **mask** (-): tensor containing indices to be masked

    Returns: context, attn
        - **context**: tensor containing the context vector from attention mechanism.
        - **attn**: tensor containing the attention (alignment) from the encoder outputs.
    """
    def __init__(self, dim: int):
        super(ScaledDotProductAttention, self).__init__()
        self.sqrt_dim = np.sqrt(dim)

    def forward(self, query: Tensor, key: Tensor, value: Tensor, mask: Optional[Tensor] = None) -> Tuple[Tensor, Tensor]:
        score = torch.bmm(query, key.transpose(1, 2)) / self.sqrt_dim

        if mask is not None:
            score.masked_fill_(mask.view(score.size()), -float('Inf'))

        attn = F.softmax(score, -1)
        context = torch.bmm(attn, value)
        return context, attn

In [7]:
class MultiHeadAttention(nn.Module):
    """
    Multi-Head Attention proposed in "Attention Is All You Need"
    Instead of performing a single attention function with d_model-dimensional keys, values, and queries,
    project the queries, keys and values h times with different, learned linear projections to d_head dimensions.
    These are concatenated and once again projected, resulting in the final values.
    Multi-head attention allows the model to jointly attend to information from different representation
    subspaces at different positions.

    MultiHead(Q, K, V) = Concat(head_1, ..., head_h) · W_o
        where head_i = Attention(Q · W_q, K · W_k, V · W_v)

    Args:
        d_model (int): The dimension of keys / values / quries (default: 512)
        num_heads (int): The number of attention heads. (default: 8)

    Inputs: query, key, value, mask
        - **query** (batch, q_len, d_model): In transformer, three different ways:
            Case 1: come from previoys decoder layer
            Case 2: come from the input embedding
            Case 3: come from the output embedding (masked)

        - **key** (batch, k_len, d_model): In transformer, three different ways:
            Case 1: come from the output of the encoder
            Case 2: come from the input embeddings
            Case 3: come from the output embedding (masked)

        - **value** (batch, v_len, d_model): In transformer, three different ways:
            Case 1: come from the output of the encoder
            Case 2: come from the input embeddings
            Case 3: come from the output embedding (masked)

        - **mask** (-): tensor containing indices to be masked

    Returns: output, attn
        - **output** (batch, output_len, dimensions): tensor containing the attended output features.
        - **attn** (batch * num_heads, v_len): tensor containing the attention (alignment) from the encoder outputs.
    """
    def __init__(self, d_model: int = 512, num_heads: int = 8):
        super(MultiHeadAttention, self).__init__()

        assert d_model % num_heads == 0, "d_model % num_heads should be zero."

        self.d_head = int(d_model / num_heads)
        self.num_heads = num_heads
        self.scaled_dot_attn = ScaledDotProductAttention(self.d_head)
        self.query_proj = nn.Linear(d_model, self.d_head * num_heads)
        self.key_proj = nn.Linear(d_model, self.d_head * num_heads)
        self.value_proj = nn.Linear(d_model, self.d_head * num_heads)

    def forward(
            self,
            query: Tensor,
            key: Tensor,
            value: Tensor,
            mask: Optional[Tensor] = None
    ) -> Tuple[Tensor, Tensor]:
        batch_size = value.size(0)

        query = self.query_proj(query).view(batch_size, -1, self.num_heads, self.d_head)  # BxQ_LENxNxD
        key = self.key_proj(key).view(batch_size, -1, self.num_heads, self.d_head)      # BxK_LENxNxD
        value = self.value_proj(value).view(batch_size, -1, self.num_heads, self.d_head)  # BxV_LENxNxD

        query = query.permute(2, 0, 1, 3).contiguous().view(batch_size * self.num_heads, -1, self.d_head)  # BNxQ_LENxD
        key = key.permute(2, 0, 1, 3).contiguous().view(batch_size * self.num_heads, -1, self.d_head)      # BNxK_LENxD
        value = value.permute(2, 0, 1, 3).contiguous().view(batch_size * self.num_heads, -1, self.d_head)  # BNxV_LENxD

        if mask is not None:
            mask = mask.unsqueeze(1).repeat(1, self.num_heads, 1, 1)  # BxNxQ_LENxK_LEN

        context, attn = self.scaled_dot_attn(query, key, value, mask)

        context = context.view(self.num_heads, batch_size, -1, self.d_head)
        context = context.permute(1, 2, 0, 3).contiguous().view(batch_size, -1, self.num_heads * self.d_head)  # BxTxND

        return context, attn


## my attention agg

In [4]:
class Agg_btw_frames(nn.Module):
    def __init__(self,dim,num_heads, num_neighs):
        super(Agg_btw_frames, self).__init__()
        self.dim = dim
        self.m_att = MultiHeadAttention(dim, num_heads)
        self.num_neighs = num_neighs
    
    def forward(self, fea_i, fea_j, batch, pos_i=None, pos_j=None):
        """
        pos_i, j: (B*N, 3)
        fea_i, fea_j: (B*N, dim)
        batch: LongTensor(B*N)
        eg: t = torch.LongTensor([0,1])
            dbatch = torch.repeat_interleave(t, 1024)
        """
        if (pos_i is not None) and (pos_j is not None):
            y_to_x_index = knn(pos_i, pos_j, self.num_neighs,batch, batch)
        else:
            y_to_x_index = knn(fea_i, fea_j, self.num_neighs,batch, batch)

        out = torch.empty_like(fea_j)
        
        for i in range(fea_j.shape[-2]):
            f1_index_i = y_to_x_index[1][y_to_x_index[0,:]==i] # f5_i 根据 pos 找到的 10个f1中的neighbors
            q_i = fea_j[i,:][None, None,:] # (1,1,dim)
            k_i = fea_i[f1_index_i,:][None, :,:] # (1,k,dim)
            v_i = fea_i[f1_index_i,:][None, :,:] # (1,k,dim)
            context_i,_ = self.m_att(q_i, k_i, v_i) # (1,1,dim)
            # print(context.shape)
            out[i,:] = context_i.view(-1, self.dim)

        return(out)

In [8]:
# batch_size = 2, num_points = 1024, num_pred = 5

bpos_1 = torch.randn(2*1024,3)
bpos_5 =torch.randn(2*1024,3)
bfea_1 = torch.randn(2*1024,24)
bfea_5 =torch.randn(2*1024,24)
t = torch.LongTensor([0,1])
dbatch = torch.repeat_interleave(t, 1024)

In [6]:
mymodel = Agg_btw_frames(dim=24, num_heads=4, num_neighs=10)
out = mymodel(bfea_1,bfea_5,dbatch,bpos_1, bpos_5)
out.shape, out

(torch.Size([2048, 24]),
 tensor([[ 0.0467,  0.2136, -0.0252,  ...,  0.1901,  0.2268,  0.1500],
         [-0.3595,  0.3060,  0.2184,  ..., -0.3934,  0.3580,  0.4118],
         [ 0.0872,  0.0186,  0.2714,  ...,  0.3095, -0.1507,  0.1483],
         ...,
         [ 0.3661, -0.0559, -0.1092,  ..., -0.1361,  0.2176,  0.0148],
         [ 0.0114, -0.1161,  0.2120,  ...,  0.1526,  0.1066, -0.1007],
         [ 0.3157,  0.1668,  0.2611,  ...,  0.0051,  0.5298,  0.0334]],
        grad_fn=<CopySlices>))

In [ ]:
for name, parameter in mymodel.named_parameters():
    print(f"Parameter name: {name}")
    print(f"Parameter shape: {parameter.shape}")
    print()

## test for GAT

In [3]:
bpos_1 = torch.randn(2*1024,3)
bpos_5 =torch.randn(2*1024,3)
bfea_1 = torch.randn(2*1024,24)
bfea_5 =torch.randn(2*1024,24)
t = torch.LongTensor([0,1])
dbatch = torch.repeat_interleave(t, 1024)

In [4]:
by_to_x_index = knn(x=bfea_1,y=bfea_5, k=10, batch_x=dbatch, batch_y=dbatch)
type(by_to_x_index), by_to_x_index.shape, by_to_x_index

(torch.Tensor,
 torch.Size([2, 20480]),
 tensor([[   0,    0,    0,  ..., 2047, 2047, 2047],
         [ 589,  256,  974,  ..., 1394, 1106, 1396]]))

In [5]:
by_to_x_index[1][by_to_x_index[0,:]==1027], by_to_x_index[1][by_to_x_index[0,:]==104]

(tensor([1545, 1418, 1847, 1516, 1093, 1475, 1817, 1197, 2011, 1827]),
 tensor([  38,  234,  992,  826,    0,  142,  469, 1002,  619,  140]))

In [6]:
bfea_1 = torch.randn(2*100,24)
bfea_5 =torch.randn(2*100,24)
t = torch.LongTensor([0,1])
dbatch = torch.repeat_interleave(t, 100)
by_to_x_index = knn(x=bfea_1,y=bfea_5, k=4, batch_x=dbatch, batch_y=dbatch)
by_to_x_index

tensor([[  0,   0,   0,  ..., 199, 199, 199],
        [ 76,  67,  30,  ..., 128, 144, 183]])

In [9]:
gat_btw_fr = GATv2Conv(in_channels=(24,24),out_channels=24, heads=2, concat=False, negative_slope=0.2, bias=False, add_self_loops=False, flow='target_to_source')
out = gat_btw_fr((bfea_5, bfea_1),by_to_x_index)
out.shape

alpha torch.Size([800, 2])
tensor([0.1248, 0.1533], grad_fn=<SelectBackward0>)
out.shape:  torch.Size([200, 2, 24])


torch.Size([200, 24])

## some tests for aggs

In [ ]:
pos_1 = torch.randn(1024,3)
pos_5 =torch.randn(1024,3)
fea_1 = torch.randn(1024,24)
fea_5 = torch.randn(1024,24)
y_to_x_index = knn(pos_1, pos_5, 10)
y_to_x_index

In [7]:
bpos_1 = torch.randn(2*1024,3)
bpos_5 =torch.randn(2*1024,3)
t = torch.LongTensor([0,1])
dbatch = torch.repeat_interleave(t, 1024)

by_to_x_index = knn(x=bpos_1,y=bpos_5, k=10, batch_x=dbatch, batch_y=dbatch)
by_to_x_index.shape, by_to_x_index




(torch.Size([2, 20480]),
 tensor([[   0,    0,    0,  ..., 2047, 2047, 2047],
         [ 821,  775,  811,  ..., 1118, 1518, 1169]]))

In [8]:
by_to_x_index[1][by_to_x_index[0,:]==1027], by_to_x_index[1][by_to_x_index[0,:]==104]

(tensor([1325, 1795, 1399, 1340, 1600, 1892, 1537, 1282, 1617, 1191]),
 tensor([158, 479, 162, 761, 632, 689, 584, 462, 872, 497]))

In [ ]:
i = 0
f1_index_i = by_to_x_index[1][by_to_x_index[0,:]==i] # f5_i 根据 pos 找到的 10个f1中的neighbors

q_i = bfea_5[i,:][None, None,:]
print(q_i.shape)
k_i = bfea_1[f1_index_i,:][None, :,:]
v_i = bfea_1[f1_index_i,:][None, :,:]
print(k_i.shape)

f_1_5_attn = ScaledDotProductAttention(dim=24)
context, attn = f_1_5_attn(q_i, k_i,v_i)
context, attn, context.shape, attn.sum()

In [ ]:
i = 0
f1_index_i = y_to_x_index[1][y_to_x_index[0,:]==i] # f5_i 根据 pos 找到的 10个f1中的neighbors

q_i = fea_5[i,:][None, None,:]
print(q_i.shape)
k_i = fea_1[f1_index_i,:][None, :,:]
v_i = fea_1[f1_index_i,:][None, :,:]
print(k_i.shape)

f_1_5_attn = ScaledDotProductAttention(dim=24)
context, attn = f_1_5_attn(q_i, k_i,v_i)
context, attn, context.shape, attn.sum()

In [ ]:
f_1_5_m_attn = MultiHeadAttention(d_model=24, num_heads=4)
m_context, m_attn = f_1_5_m_attn(q_i, k_i,v_i)
m_context, m_attn, m_context.shape, m_attn.sum()

In [ ]:
for name, parameter in f_1_5_m_attn.named_parameters():
    print(f"Parameter name: {name}")
    print(f"Parameter shape: {parameter.shape}")
    print()

## go on

In [ ]:
# input
pc_seq = []
pc_pos_seq = []
for _ in range(5):
    t = torch.randn(2,1024,24)
    f = torch.randn(2,1024,3)
    pc_seq.append(t)
    pc_pos_seq.append(f)

In [ ]:
fea_seq = []
for pc in pc_seq:
    #global_f_extractor to be updated
    pc_global_fea, _ = torch.max(pc, dim=-2)
    fea_seq.append(pc_global_fea)

q = fea_seq[-1][:, None, :] # (B, 1, C)
k = torch.stack(fea_seq[0:-1],dim=-2) # (B, T-1, C)
v = torch.stack(fea_seq[0:-1],dim=-2)  # (B, T-1, C)
print(q.shape, k.shape, v.shape)

mat = ScaledDotProductAttention(24)
context, att = mat(q,k,v)
context.shape, context, att.shape, att

In [ ]:
local_agg_model = Agg_btw_frames(dim=24, num_heads=4, num_neighs=10)
batch_size = 2

t = torch.LongTensor(range(batch_size))
dbatch = torch.repeat_interleave(t, 1024)

res = torch.empty(batch_size,len(pc_pos_seq)-1, 1024, 24) #(2,4,1024,24)

for i in range(len(pc_pos_seq)-1):
    out = local_agg_model(pc_seq[i].view(-1,24), pc_seq[-1].view(-1,24), dbatch,pc_pos_seq[i].view(-1,3), pc_pos_seq[-1].view(-1,3))
    print(out.shape)
    res[:,i,:,:] = out.view(batch_size,1024, 24)


In [ ]:
# print(att.shape)
# print(att)
fr_weight = att.permute(0,2,1).unsqueeze(-1)
# print(fr_weight.shape)
# print(fr_weight)
weighted_res = torch.sum(torch.mul(res, fr_weight),dim=1)

In [ ]:
res.shape

In [ ]:
weighted_res.shape

In [ ]:
pc_seq[-1].shape, weighted_res.shape

In [ ]:
pred_fea = torch.cat([pc_seq[-1].unsqueeze(1), weighted_res.unsqueeze(1)], dim=1)

In [ ]:
pred_fea.shape

In [ ]:

# 定义输入张量 t 和权重张量 weight
t = torch.randn(2, 4, 1024, 24)
weight = torch.randn(2, 1, 4)

# 将权重张量的形状扩展为 (2, 4, 1, 1)
weight_expanded = weight.unsqueeze(-1).unsqueeze(-1).squeeze(1)

weighted = torch.mul(t, weight_expanded)
print(weighted.shape)  # 输出 (2, 4, 1024, 24)

# 对帧的维度求和，得到加权求和的结果
aggregated = torch.sum(weighted, dim=1)

print(aggregated.shape)  # 输出 (2, 1024, 24)


In [ ]:
# 根据 global feature 生成 的 frame_attention 聚合 res 中的每个frame
frame_att = att

## encoder

In [ ]:
encoder_args = {
    'channels': 24,
    'k': 20,
    'dilations': (1,2),
    'n_idgcn_blocks':2,
    'n_dgcn_blocks':2,
    'radio': 0.25,
}

In [ ]:
encoder_kargs = {

    'use_radius_graph': False,
    'use_bottleneck': True,
    'use_pooling': True,
    'use_residual': True,
    'conv': 'edge',
    'pool_type': 'mean',
    'dynamic': False,
    'hierarchical': True,
    'ratio': 0.25,
}

In [ ]:
from pugcn_lib.feature_extractor import InceptionFeatureExtractor
pc_encoder = InceptionFeatureExtractor(**encoder_args, **encoder_kargs)

In [ ]:
# some config
num_points = 1024*4

batch_size = 2
num_frames = 5
global_fea_dim = 24
local_fea_dim = 24
num_heads = 4
num_neighs = 10

In [ ]:
# batch tensor
t = torch.LongTensor(range(batch_size))
in_batch = torch.repeat_interleave(t, num_points)
encoder_batch = torch.repeat_interleave(t, int(num_points * encoder_kargs['ratio']))
in_batch.shape, encoder_batch.shape

In [ ]:
input_pc_seq = []
for _ in range(num_frames):
    pc = torch.randn(batch_size*num_points,3)
    input_pc_seq.append(pc)

In [ ]:
pc_encoded_seq = []
global_fea_seq = []
for i,pc in enumerate(input_pc_seq):
    pc_encoded = pc_encoder(x=pc, pos = pc, batch=in_batch, return_index = False)
    print(pc_encoded.shape)
    pc_encoded_seq.append(pc_encoded)

    #TODO: some node level projection before pooling
    pc_pooled = global_mean_pool(pc_encoded, batch=encoder_batch).unsqueeze(1)
    print(pc_pooled.shape)
    global_fea_seq.append(pc_pooled)


In [ ]:
q = global_fea_seq[-1] # (B, 1, C)
k = torch.stack(global_fea_seq[0:-1],dim=-2).squeeze(1) # (B, T-1, C)
v = torch.stack(global_fea_seq[0:-1],dim=-2).squeeze(1)  # (B, T-1, C)

print(q.shape, k.shape, v.shape)

mat = ScaledDotProductAttention(global_fea_dim)
_, frame_level_att = mat(q,k,v)
frame_level_att.shape, frame_level_att


In [ ]:
local_agg_model = Agg_btw_frames(dim=local_fea_dim, num_heads=num_heads, num_neighs=num_neighs)

res = torch.empty(batch_size,len(pc_encoded_seq)-1, int(num_points * encoder_kargs['ratio']), local_fea_dim)
print(res.shape)

for i, pc_encoded in enumerate(pc_encoded_seq[:-1]):
    out = local_agg_model(pc_encoded, pc_encoded_seq[-1], encoder_batch)
    print(out.shape)
    res[:,i,:,:] = out.view(batch_size,int(num_points * encoder_kargs['ratio']), local_fea_dim)

In [ ]:
fr_weight = frame_level_att.permute(0,2,1).unsqueeze(-1)
weighted_res = torch.sum(torch.mul(res, fr_weight),dim=1)
fr_weight.shape, weighted_res.shape, 

In [ ]:
time_series_output = torch.cat([pc_encoded_seq[-1], weighted_res.view(-1,local_fea_dim)], dim=-1)
time_series_output.shape

In [ ]:
# what decoder see:
time_series_output.shape, encoder_batch.shape

## upsampler and refiner

In [49]:
from pugcn_lib.upsample import GeneralUpsampler
from pugcn_lib.models import Refiner

In [50]:
tm_out = torch.randn(4096, 256)
tm_pos = torch.randn(4096, 3)

t = torch.LongTensor([0,1,2,3])
encoder_batch = torch.repeat_interleave(t, 1024)

tm_out.shape, encoder_batch.shape

(torch.Size([4096, 256]), torch.Size([4096]))

In [51]:
upsampler_kargs = {
    "upsampler": "nodeshuffle",
    "in_channels": 256,
    "out_channels": 64,
    "k": 40,
    "r": 16, 
    "conv":"gatv2",
    "heads":4
}

In [52]:
upsampler = GeneralUpsampler(**upsampler_kargs)
upsampled_pc, up_batch = upsampler(x=tm_out,pos=tm_pos, batch=encoder_batch, return_batch=True)
upsampled_pc.shape, up_batch.shape

(torch.Size([65536, 64]), torch.Size([65536]))

In [53]:
reconstructor =  torch.nn.Sequential(
            torch.nn.Linear(64, 24),
            torch.nn.LeakyReLU(),
            torch.nn.Linear(24, 3),)
pc_reconstructed = reconstructor(upsampled_pc)
pc_reconstructed.shape

torch.Size([65536, 3])

In [58]:
refiner_kargs = {
    "in_channels": 64,
    "out_channels": 3,
    "k": 30,
    "dilations": (1,2),
    "add_points": True
}

In [59]:
pc_refiner = Refiner(**refiner_kargs)
print(upsampled_pc.shape)
print(pc_reconstructed.shape)
print(up_batch.shape)

pc_f1 = pc_refiner(x=upsampled_pc, pos=pc_reconstructed, batch=up_batch)
pc_f1.shape

torch.Size([65536, 64])
torch.Size([65536, 3])
torch.Size([65536])


torch.Size([65536, 3])

## some test

In [ ]:
t_pc = torch.randn(batch_size*num_points,3)
t_pc_encoded = pc_encoder(x=t_pc, pos = t_pc, batch=in_batch, return_index = False)
t_pc_encoded.shape

#TODO: some projection before pooling

pc_pooled = global_mean_pool(t_pc_encoded, batch=encoder_batch)

pc_pooled.shape, pc_pooled

In [ ]:
a = torch.randn(2048,24)
b = torch.randn(2048,24)


t_out = local_agg_model(a,b,encoder_batch)
t_out.shape

In [ ]:
a = torch.randn(2048,24)
b = torch.randn(2048,24)
pa = torch.randn(2048,3)
pb = torch.randn(2048,3)


t_out = local_agg_model(a,b,encoder_batch,pa,pb)
t_out.shape

## complete model

In [2]:
from dataset.kitti_dataset import KittiDataset
kdata = "./kittiraw/dataset/sequences"
pc_seq_dataset = KittiDataset(root=kdata, npoints=4096, input_num=5, pred_num=5, seqs=['01'])

In [3]:
from torch.utils.data import DataLoader

In [4]:
len(pc_seq_dataset)

110

In [5]:
input_pc, gt_out_pc = pc_seq_dataset[2]
print(input_pc.shape)
print(gt_out_pc.shape)

torch.Size([5, 4096, 3])
torch.Size([5, 4096, 3])


In [21]:
class KittiLoader(DataLoader):
    def __init__(self, dataset, batch_size):
        super(KittiLoader, self).__init__(dataset, batch_size=batch_size, collate_fn=self.collate_fn)

    def collate_fn(self, batch):
        a_list, b_list = zip(*batch)

        a_batch = torch.cat(a_list, dim=1)
        b_batch = torch.cat(b_list, dim=1)

        # concatenated_batch = torch.cat((a_batch, b_batch), dim=1)

        return a_batch, b_batch

In [24]:
k_loader = KittiLoader(pc_seq_dataset, batch_size=2)


In [25]:
input_pc_seq , gt_pc_seq = next(iter(k_loader))
input_pc_seq.shape, gt_pc_seq.shape

(torch.Size([5, 8192, 3]), torch.Size([5, 8192, 3]))

In [ ]:
assert(input_pc_seq.shape == torch.Size([5, 4096, 3])), "input_pc_seq shape error"

In [ ]:
encoder_args = {'channels': 24, 'k': 20, 'dilations': (1,2), 'n_idgcn_blocks':2, 'n_dgcn_blocks':2, 'radio': 0.25}
encoder_kargs = { 'use_radius_graph': False, 'use_bottleneck': True, 'use_pooling': True, 'use_residual': True, 'conv': 'edge', 'pool_type': 'mean',
'dynamic': False, 'hierarchical': True,}
dim_args = {'global_fea_dim': 120 }
att_args = {'num_heads': 4, 'num_neighs': 10}
upsampler_args = { "upsampler": "nodeshuffle", "in_channels": 48, "out_channels": 24, "k": 20, "r": 4}
refiner_kargs = { "in_channels": 24, "out_channels": 3, "k": 20, "dilations": (1,2), "add_points": True }

In [ ]:
from models.time_series_models import PC_forecasting_model_0_0

model = PC_forecasting_model_0_0(encoder_args, encoder_kargs, dim_args, att_args, upsampler_args, refiner_kargs)
batch_size = 2
k_loader = KittiLoader(pc_seq_dataset, batch_size=2)
input_pc_seq = next(iter(k_loader))[0]

pred_pc_seq = model(input_pc_seq, batch_size,num_pred=5, device='cpu')
for i, pc in enumerate(pred_pc_seq):
    print(i, pc.shape)


## point_cloud vis

xvfb missed! cannot use open3d